In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'
}

In [87]:
def parse_page(url):
    response = requests.get(url, headers=headers)
    text = response.content.decode('utf-8')
    soup = BeautifulSoup(text, 'html5lib')
    conMidtab = soup.find_all('div', class_='conMidtab')[1]
    tables = conMidtab.find_all('table')
    for table in tables:
        trs = table.find_all('tr')[2:]
        for index, tr in enumerate(trs):
            tds = tr.find_all('td')
            if index == 0:
                city_td = tds[1]
            else:
                city_td = tds[0]
            city = list(city_td.stripped_strings)[0]
            temp_td = tds[-2]
            temp = list(temp_td.stripped_strings)[0]
            ALL_DATA.append({'city':city, 'min_temp':int(temp)})

In [88]:
ALL_DATA = []
def main():
    area_list = ['hb','db','hd','hz','hn','xb','xn','gat']
    for area in area_list:
        url = 'http://www.weather.com.cn/textFC/{}.shtml'.format(area)
        parse_page(url)  
    
if __name__ == '__main__':
    main() 

In [91]:
SORTED_ALL_DATA = sorted(ALL_DATA, key=lambda x: x['min_temp'])

In [113]:
top_10 = SORTED_ALL_DATA[:10]

In [103]:
from pyecharts.charts import Bar
from pyecharts import options as opts

In [114]:
cities = list(map(lambda x:x['city'], top_10))
min_temp = list(map(lambda x:x['min_temp'], top_10))

In [115]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [116]:
bar = (Bar()
      .add_xaxis(cities)
      .add_yaxis('温度', min_temp)
      .set_global_opts(title_opts=opts.TitleOpts(title="5月9日气温最低的前十名城市"))
)
bar.load_javascript()

In [117]:
bar.render_notebook()